## EIGENMIKE CONFIGURATION. 
### Sources are plotted at the same coordinates regardless of changes to the radius of the microphone array. Distances between the microphone array and the sources are not preserved with changes to the radius of the microphone array. 

In [ ]:
### VISUALIZE 
# Zoom into the top of box to see contents 

import trimesh
import numpy as np
from rlr_audio_propagation import Config, Context, ChannelLayout, ChannelLayoutType
import matplotlib.pyplot as plt
import soundfile as sf

width, depth, height = 5.0, 5.0, 2.0 
vertices = np.array([
    [0, 0, 0], [width, 0, 0], [width, depth, 0], [0, depth, 0],
    [0, 0, height], [width, 0, height], [width, depth, height], [0, depth, height]
])
faces = np.array([
    [0, 1, 2], [0, 2, 3], [4, 5, 6], [4, 6, 7],
    [0, 4, 7], [0, 7, 3], [1, 5, 6], [1, 6, 2],
    [3, 2, 6], [3, 6, 7], [0, 1, 5], [0, 5, 4]
])

def spherical_to_cartesian(r, theta, phi):
    theta_rad = np.radians(theta)
    phi_rad = np.radians(phi)
    x = r * np.sin(theta_rad) * np.cos(phi_rad)
    y = r * np.sin(theta_rad) * np.sin(phi_rad)
    z = r * np.cos(theta_rad)
    return x, y, z

eigenmike_raw = {
    # colatitude, azimuth, radius
    # (degrees, degrees, meters)
    "1": [69, 0, 0.16],
    "2": [90, 32, 0.16],
    "3": [111, 0, 0.16],
    "4": [90, 328, 0.16],
    "5": [32, 0, 0.16],
    "6": [55, 45, 0.16],
    "7": [90, 69, 0.16],
    "8": [125, 45, 0.16],
    "9": [148, 0, 0.16],
    "10": [125, 315, 0.16],
    "11": [90, 291, 0.16],
    "12": [55, 315, 0.16],
    "13": [21, 91, 0.16],
    "14": [58, 90, 0.16],
    "15": [121, 90, 0.16],
    "16": [159, 89, 0.16],
    "17": [69, 180, 0.16],
    "18": [90, 212, 0.16],
    "19": [111, 180, 0.16],
    "20": [90, 148, 0.16],
    "21": [32, 180, 0.16],
    "22": [55, 225, 0.16],
    "23": [90, 249, 0.16],
    "24": [125, 225, 0.16],
    "25": [148, 180, 0.16],
    "26": [125, 135, 0.16],
    "27": [90, 111, 0.16],
    "28": [55, 135, 0.16],
    "29": [21, 269, 0.16],
    "30": [58, 270, 0.16],
    "31": [122, 270, 0.16],
    "32": [159, 271, 0.16],
}

mic_positions = [(eigenmike_raw[str(i)][0], eigenmike_raw[str(i)][1]) for i in range(1, 33)]
mic_radius = 0.06 

mic_cartesian = [spherical_to_cartesian(mic_radius, theta, phi) for theta, phi in mic_positions]
room_center = [width/2, depth/2, height/2]

mic_meshes = []
mic_absolute_positions = []
for i, (x, y, z) in enumerate(mic_cartesian):
    mic_pos = [room_center[0] + x, room_center[1] + y, room_center[2] + z]
    mic_absolute_positions.append(mic_pos)
    mic_mesh = trimesh.creation.icosphere(radius=0.02, subdivisions=2) 
    mic_mesh.apply_translation(mic_pos)
    mic_mesh.visual.face_colors = [0, 0, 255, 255] 
    mic_meshes.append(mic_mesh)

# Set up config with reverberation
cfg = Config()
cfg.indirect_ray_count = 50000 
cfg.indirect_ray_depth = 25  
cfg.source_ray_count = 50000 
cfg.source_ray_depth = 25 
cfg.max_diffraction_order = 1
cfg.direct_ray_count = 8000 
cfg.max_ir_length = 1.5 
cfg.mesh_simplification = False

ctx = Context(cfg)
ctx.add_object()
ctx.set_object_position(0, [0, 0, 0])
ctx.add_mesh_vertices(vertices.flatten().tolist())
ctx.add_mesh_indices(faces.flatten().tolist(), 3, "default")
ctx.finalize_object_mesh(0)

for i, mic_pos in enumerate(mic_absolute_positions):
    ctx.add_listener(ChannelLayout(ChannelLayoutType.Mono, 1))
    ctx.set_listener_position(i, mic_pos)

# make 5x5x5 grid of source positions, excluding center
grid_size = 5
x_positions = np.linspace(0.1 * width, 0.9 * width, grid_size)
y_positions = np.linspace(0.1 * depth, 0.9 * depth, grid_size)
z_positions = np.linspace(0.1 * height, 0.9 * height, grid_size)
source_positions = []
for x in x_positions:
    for y in y_positions:
        for z in z_positions:
            # Check that position is not the center
            if not np.allclose([x, y, z], room_center, atol=mic_radius):
                source_positions.append([x, y, z])

box_mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
for i, position in enumerate(source_positions):
    ctx.add_source()
    ctx.set_source_position(i, position)
    
    source_sphere = trimesh.creation.icosphere(radius=0.05, subdivisions=2)
    source_sphere.apply_translation(position)
    source_sphere.visual.face_colors = [0, 255, 0, 255] 
    box_mesh = trimesh.util.concatenate([box_mesh, source_sphere])

combined_mesh = trimesh.util.concatenate([box_mesh] + mic_meshes)
print(f"Is the mesh watertight? {combined_mesh.is_watertight}")
print(f"Number of sources: {len(source_positions)}")
print(f"Number of microphones: {len(mic_meshes)}")

combined_mesh.show()

In [ ]:
### DATA GENERATION 

import os
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from rlr_audio_propagation import Config, Context, ChannelLayout, ChannelLayoutType

width, depth, height = 5.0, 5.0, 2.0 
vertices = np.array([
    [0, 0, 0], [width, 0, 0], [width, depth, 0], [0, depth, 0],
    [0, 0, height], [width, 0, height], [width, depth, height], [0, depth, height]
])
faces = np.array([
    [0, 1, 2], [0, 2, 3], [4, 5, 6], [4, 6, 7],
    [0, 4, 7], [0, 7, 3], [1, 5, 6], [1, 6, 2],
    [3, 2, 6], [3, 6, 7], [0, 1, 5], [0, 5, 4]
])

def spherical_to_cartesian(r, theta, phi):
    theta_rad = np.radians(theta)
    phi_rad = np.radians(phi)
    x = r * np.sin(theta_rad) * np.cos(phi_rad)
    y = r * np.sin(theta_rad) * np.sin(phi_rad)
    z = r * np.cos(theta_rad)
    return x, y, z

eigenmike_raw = {
    # colatitude, azimuth, radius
    # (degrees, degrees, meters)
    "1": [69, 0, 0.16],
    "2": [90, 32, 0.16],
    "3": [111, 0, 0.16],
    "4": [90, 328, 0.16],
    "5": [32, 0, 0.16],
    "6": [55, 45, 0.16],
    "7": [90, 69, 0.16],
    "8": [125, 45, 0.16],
    "9": [148, 0, 0.16],
    "10": [125, 315, 0.16],
    "11": [90, 291, 0.16],
    "12": [55, 315, 0.16],
    "13": [21, 91, 0.16],
    "14": [58, 90, 0.16],
    "15": [121, 90, 0.16],
    "16": [159, 89, 0.16],
    "17": [69, 180, 0.16],
    "18": [90, 212, 0.16],
    "19": [111, 180, 0.16],
    "20": [90, 148, 0.16],
    "21": [32, 180, 0.16],
    "22": [55, 225, 0.16],
    "23": [90, 249, 0.16],
    "24": [125, 225, 0.16],
    "25": [148, 180, 0.16],
    "26": [125, 135, 0.16],
    "27": [90, 111, 0.16],
    "28": [55, 135, 0.16],
    "29": [21, 269, 0.16],
    "30": [58, 270, 0.16],
    "31": [122, 270, 0.16],
    "32": [159, 271, 0.16],
}

mic_positions = [(eigenmike_raw[str(i)][0], eigenmike_raw[str(i)][1]) for i in range(1, 33)]
mic_radius = 0.16  

mic_cartesian = [spherical_to_cartesian(mic_radius, theta, phi) for theta, phi in mic_positions]
room_center = [width/2, depth/2, height/2]

mic_absolute_positions = []
for x, y, z in mic_cartesian:
    mic_pos = [room_center[0] + x, room_center[1] + y, room_center[2] + z]
    mic_absolute_positions.append(mic_pos)

__TETRA_CHANS_IN_EM32__ = [5, 9, 25, 21]

wav_dir = f"Eigenmike_wavs_{mic_radius}"
plot_dir = f"Eigenmike_plots_{mic_radius}"
os.makedirs(wav_dir, exist_ok=True)
os.makedirs(plot_dir, exist_ok=True)
print(f"Created directories: {wav_dir} and {plot_dir}")

cfg = Config()
cfg.indirect_ray_count = 50000 
cfg.indirect_ray_depth = 25  
cfg.source_ray_count = 50000 
cfg.source_ray_depth = 25 
cfg.max_diffraction_order = 1
cfg.direct_ray_count = 8000 
cfg.max_ir_length = 1.5 
cfg.mesh_simplification = False

ctx = Context(cfg)
ctx.add_object()
ctx.set_object_position(0, [0, 0, 0])
ctx.add_mesh_vertices(vertices.flatten().tolist())
ctx.add_mesh_indices(faces.flatten().tolist(), 3, "default")
ctx.finalize_object_mesh(0)

# Add listeners
for i, mic_index in enumerate(__TETRA_CHANS_IN_EM32__):
    ctx.add_listener(ChannelLayout(ChannelLayoutType.Mono, 1))
    ctx.set_listener_position(i, mic_absolute_positions[mic_index])

ctx.add_source()

# make 5x5x5 grid of source positions, excluding center
grid_size = 5
x_positions = np.linspace(0.1 * width, 0.9 * width, grid_size)
y_positions = np.linspace(0.1 * depth, 0.9 * depth, grid_size)
z_positions = np.linspace(0.1 * height, 0.9 * height, grid_size)
source_positions = []
for x in x_positions:
    for y in y_positions:
        for z in z_positions:
            # Check that position is not the center
            if not np.allclose([x, y, z], room_center, atol=mic_radius):
                source_positions.append([x, y, z])

for source_index, source_position in enumerate(source_positions):
    print(f"\nProcessing source {source_index + 1} at position {source_position}")
    
    # Calculate source's position relative to room center for file name
    relative_position = np.array(source_position) - np.array(room_center)
    x, y, z = relative_position
    coord_filename = f"{source_index:03d}_{x:.2f}_{y:.2f}_{z:.2f}"
    output_filename = os.path.join(wav_dir, f"{coord_filename}.wav")
    plot_filename = os.path.join(plot_dir, f"{coord_filename}.png")
    
    print(f"Output WAV file: {output_filename}")
    print(f"Output plot file: {plot_filename}")
    
    ir_all_mics = []
    ir_lengths = []
    
    # Process one microphone at a time
    for i, mic_index in enumerate(__TETRA_CHANS_IN_EM32__):
        print(f"  Processing microphone {mic_index}")
        
        ctx.set_source_position(0, source_position)
        
        print(f"    Simulating...")
        ctx.simulate()
        
        # Write each mic to 1 channel of IR 
        channel = np.array(ctx.get_ir_channel(i, 0, 0))
        ir_all_mics.append(channel)
        ir_lengths.append(len(channel))
        print(f"    IR length: {len(channel)} samples")
    
    max_length = max(ir_lengths)
    print(f"  Max IR length: {max_length} samples")
    
    # Pad shorter IRs with zeros to match longest IR
    ir_all_mics_padded = [np.pad(ir, (0, max_length - len(ir)), 'constant') for ir in ir_all_mics]
    ir_all_mics = np.array(ir_all_mics_padded)
    
    sample_rate = int(cfg.sample_rate)
    print(f"  Writing WAV file with sample rate: {sample_rate} Hz")
    sf.write(output_filename, ir_all_mics.T, sample_rate)
    
    print("  Generating plot...")
    plt.figure(figsize=(15, 10))
    for i, mic_index in enumerate(__TETRA_CHANS_IN_EM32__):
        plt.subplot(4, 1, i+1)
        plt.plot(ir_all_mics[i])
        plt.title(f'Microphone {mic_index}')
        plt.ylim([-1, 1])
        plt.ylabel('Amplitude')
        plt.grid(True)
    plt.xlabel('Sample')
    plt.suptitle(f'Room Impulse Response - Tetrahedral Microphone Channels\nSource {source_index:03d}: {x:.2f}, {y:.2f}, {z:.2f}', fontsize=16)
    plt.tight_layout()
    plt.savefig(plot_filename)
    plt.close()
    print("  Plot saved.")

print(f"\nAll Room Impulse Responses saved in {wav_dir}")
print(f"All plots saved in {plot_dir}")